# Introduction to the NCAS CF Data Tools, cf-python and cf-plot

***

## Setting up

**In this section we set up this Notebook, import the libraries and check the data we will work with, ready to use the libraries within this notebook.**

Run some set up for nice outputs in this Jupyter Notebook (not required in interactive Python or a script):

In [ ]:
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

Import cf-python and cf-plot:

In [ ]:
import cfplot as cfp
import cf

Inspect the versions of cf-python and cf-plot and the version of the CF Conventions those are matched to:

In [ ]:
print("cf-python version is:", cf.__version__)
print("cf-plot version is:", cfp.__version__)
print("CF Conventions version is:", cf.CF())

<div class="alert alert-block alert-info">
<i>Note:</i> you can work with data compliant by any other version of the CF Conventions, or without (much) compliance, but the CF Conventions version gives the maximum version that these versions of the tools understand the features of.
</div>

Finally, see what datasets we have to explore:

<div class="alert alert-block alert-info">
<i>Note:</i> in a Jupyter Notebook, '!' preceeeds a shell command, so this is a terminal command and not Python
</div>

In [ ]:
!ls ../ncas_data

***

## 6. Changing the underlying grid of data through regridding

**In this section we demonstrate how to change the underlying grid of the data to another grid which could be a higher- or lower- resolution one, or a completely different grid, which is called regridding or interpolation, and indicate various options cf-python supports for doing this.**

### a) Getting a _source_ field ready to regrid

We read in a precipitation field and inspect it:

In [ ]:
higher_res_field = cf.read("../ncas_data/precip_2010.nc")[0]
print(higher_res_field)

To get a feel for the data we have, let's view a basic subspace as a contour plot. Note we use the `blockfill` argument set to `True` which changes the contour plot to plot the data on a cellular basis rather than filled as smoothed contours (whether or not the contour lines are set to be shown from the `lines` argument, in this case we turn those off too):

In [ ]:
cfp.mapset()  # reset
cfp.cscale("precip_11lev")  # using a colour scale good for precipitation data
cfp.con(higher_res_field[0], blockfill=True, lines=False)

### b) Getting the _destination_ field: another field in order to regrid the previous _onto its grid_

Now we read in another precipitation field and inspect it. The key thing to note is that it is lower resolution than the previous one we read:

In [ ]:
lower_res_field = cf.read("../ncas_data/model_precip_DJF_means_low_res.nc")[0]
print(lower_res_field)

Again, get a feel for the data we have from a view of a basic subspace, to confirm that it is lower resolution (notice the larger cell blocks than before):

In [ ]:
cfp.cscale("precip_11lev")
cfp.con(lower_res_field[0], blockfill=True, lines=False)

### c) Performing the regrid operation from the source to the destination fields

Now the key step after the previous setup: regridding the first (*source*) field to the grid of the second (*destination*) field. We use the `regrids` method of cf-python to do this, where the `s` in the name stands for spherical for spherical regridding, as opposed to Cartesian regridding, also possible with cf-python.

We are going to do two calls, demonstrating two different methods of interpolation, namely the `patch` and `conservative` methods. Assign these to variables so we can compare them next:

In [ ]:
patch_regridded_field = higher_res_field.regrids(lower_res_field, method="patch")
conservative_regridded_field = higher_res_field.regrids(lower_res_field, method="conservative")

These two methods do not give the same results!

In [ ]:
patch_regridded_field.equals(conservative_regridded_field)

Recall how to do field arithmetic. We use subtraction to indicate the difference between the two regridded outcomes which differ only by the interpolation method used to regrid:

In [ ]:
difference_field = patch_regridded_field - conservative_regridded_field

### d) Finally, some more advanced cf-plot plotting to compare the source, destination, and regridded results

As with the previous section, let's end by doing more advanced cf-plot plotting to show all of the relevant fields to help us to understand what the regridding did.

Again we are using `gopen` and `gclose` wrapped around our calls to plot. This time we use `gpos` function calls which tell cf-plot exactly where we want to place each plot on the overall canvas. In this case, from our `x` and `y` extrema inputs `xmin`, `xmax`, `ymin` and `ymax`, we are building an effective 4 x 4 grid to place the four contour plots we will show.

There is a lot to take in here, but the main thing is to understand that we use `con` to generate four contour plots corresponding to different stages of our regridding, telling cf-plot to place them at one of the four places in a 4 x 4 grid, with some customisation of colour scales and colour bar levels.

Note here we are taking the [0] index subspace of all of the fields to plot, but we can use any other subspace of the data to view instead:

In [ ]:
cfp.cscale("viridis")  # reset cmap to perceptually uniform scale
cfp.gopen(rows=2, columns=2, figsize=[7, 15])  #, bottom=0.1, top=0.85)

# Configure first three plots
cfp.mapset()  # reset from previous plots in the notebook - standalone don't need this
cfp.levs(min=0, max=500, step=50)

# First three plots
cfp.gpos(xmin=0.1, xmax=0.5, ymin=0.55, ymax=1.0)
cfp.cscale("precip_11lev")
cfp.con(
    higher_res_field[0], blockfill=True, lines=False,
    title="Precipitation field\nbefore regridding",
)
cfp.gpos(xmin=0.55, xmax=0.95, ymin=0.55, ymax=1.0)
cfp.con(
    patch_regridded_field[0], blockfill=True, lines=False,
    title="...and after regridding with\nthe patch recovery method",
)
cfp.gpos(xmin=0.1, xmax=0.5, ymin=0.1, ymax=0.55)
cfp.con(
    conservative_regridded_field[0], blockfill=True, lines=False,
    title="...and after regridding with\nthe conservative method",
)
cfp.gpos(xmin=0.55, xmax=0.95, ymin=0.1, ymax=0.55)

# This final plot is showing the diff so is a bit different, apply some cf-plot plotting configuration
cfp.levs()
cfp.cscale("BlueDarkRed18")  # is a diff so use a diverging colour map
cfp.con(
    difference_field[0], blockfill=True, lines=False,
    title="Difference between the two\nregridding methods is:",
)
cfp.gclose()

More generally, using regridding with cf-python you can convert from one grid to another very different grid, e.g. from a tripolar to a regular or rotated latitude-longitude grid or vice-versa, including to/from unstructured grids (UGRID grids).

***

## Conclusion and recap of learning objectives

The NCAS CF Data Tools are a suite of Python libraries which are designed to facilitate working with data for research in the earth sciences and aligned domains. We learnt today about the cf-python (https://ncas-cms.github.io/cf-python/) and cf-plot (https://ncas-cms.github.io/cf-plot/build/). The 'cf' in the names of the NCAS CF Data Tools corresponds to the CF Conventions, a metadata standard becoming the de-facto convention across geoscience to cover the description of data so that sharing and intercomparison is simpler.

Our **learning aim** was to be able to use the NCAS CF Data Tools Python libraries, namely cf-python and cf-plot to process, analyse and visualise netCDF and PP datasets, whilst appreciating the context and 'unique selling point' of the libraries as being built to use the CF Conventions, a metadata standard for earth science data, to make it simpler to do what you want to do with the datasets, by working on top of a Data Model for CF.

For our **learning objectives**, we practiced using cf-python and cf-plot to:

* read dataset(s) and view the (meta)data at different detail levels;
* edit the (meta)data and write out the edited version to file;
* reduce datasets by subspacing and collapsing;
* visualise datasets as contour and vector plots;
* analyse data: applying mathematical and statistical operations and plotting trends;
* change the underlying grid of data through regridding.

***

## Where to find more information and resources on the NCAS CF Data Tools

Here are some links relating to the NCAS CF Data Tools and this training.

* This training, with further material, is hosted online and there are instructions for setting up the environment so you can work through it in your own time: https://github.com/NCAS-CMS/cf-tools-training.
* The cf-python documentation lives at https://ncas-cms.github.io/cf-python/.
* The cf-python code lives on GitHub at https://github.com/NCAS-CMS/cf-python. There is an Issue Tracker to report queries or questions at https://github.com/NCAS-CMS/cf-python/issues.
* The cf-plot documentation lives at https://ncas-cms.github.io/cf-plot/build/.
* The cf-plot code lives on GitHub at https://github.com/NCAS-CMS/cf-plot. There is an Issue Tracker to report queries or questions at https://github.com/NCAS-CMS/cf-plot/issues.
* There is a technical presentation about the NCAS CF Data Tools avaialble from https://hps.vi4io.org/_media/events/2020/summer-school-cfnetcdf.pdf.
* The website of the CF Conventions can be found at https://cfconventions.org/.
* The landing page for training into the CF Conventions is found here within the website above: https://cfconventions.org/Training/.

If you have any queries after this course, please either use the Issue Trackers linked above or you can email me at: sadie.bartholomew@ncas.ac.uk.

***